## MBA em Ciência de Dados
# Redes Neurais e Aprendizado Profundo

## <span style="color:darkred">Avaliação Final - 2024 : Enuncaido</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

Nessa avaliação será utilizado o dataset `imdb-reviews-pt-br.csv` que contém avaliações de filmes realizadas no site IMDB, com textos em português e em inglês. O alvo do dataset é a coluna `sentiment` que contém o sentimento relacionado à avaliação: positivo ou negativo.

Vamos explorar um sentence embedding pré-treinado para multiplos idiomas, e avaliar redes neurais comparando-as com relação aos textos em inglês e português.

As tarefas a realizar são as seguintes:

Preparação:
1. **Instalar o pacote** `sentence_transformers` e **carregar o modelo** 'stsb-xlm-r-multilingual' para geração de sentence embeddings, conforme mostrado no código abaixo.
2. **Carregar a base de dados** e obter uma amostra de 12 mil linhas (use o método `sample` do dataframe pandas).
3. **Gerar os embeddings** usando o método encode, para os textos em português e em inglês. Ver https://www.sbert.net/examples/applications/computing-embeddings/README.html para mais detalhes. OBS: pode demorar um pouco
4. **Separar dados** 75% para treinamento, 25% para teste usando `train_test_split`, sendo a mesma partição para ingles e portugues.

Modelos:
1.  (1,0 pt) **Modelo A**: projete e treine uma rede neural profunda densa, utilizando como entrada os embeddings pré-treinados dos dados em **portugues** para análise de sentimento (classificação binária).<br>
    * A arquitetura deve ter portanto as seguintes camadas:
        * entrada
        * normalização em batch
        * densa 768 neurônios, relu
        * densa 512 neurônios, relu
        * densa 256 neurônios, ativação linear
        * normalização em batch
        * ativação relu
        * dropout 0.375
        * densa 1 neurônio, sigmoide
    * Utilizar Adam com taxa de aprendizado inicial de 0.0007 e com decaimento em todas as épocas exponencial a -0.09
    * Treinar com perda entropia cruzada por 15 épocas com batch size 20
    * Compute como métricas, além da perda, a área sob a curva ROC (AUC ROC) e a Acurácia Binária (ver https://www.tensorflow.org/api_docs/python/tf/keras/metrics)<br><br>

2.  (1,0 pt) **Modelo B**: projete e treine uma rede neural profunda densa, utilizando como entrada os *textos tokenizados* em **português** para análise de sentimento (classificação binária).<br>
    * A arquitetura deve ter portanto as seguintes camadas:
        * entrada
        * camada embedding com max_words=5000 e dimensão do embedding de tamanho 386
        * camada convolucional 1d com 64 neuronios de tamanho 2 e padding='same', ativação relu
        * camada LSTM 512 neurônios, sem especificar ativação (manter a default)
        * densa 256 neurônios, ativação linear
        * normalização em batch
        * ativação relu
        * dropout 0.375
        * densa 1 neurônio, sigmoide
    * Utilizar os mesmos hiper-parâmetros de otimização, treinamento e métricas do modelo anterior<br><br>


3. (2,0 pt) **Avalie as rede neurais de classificação** (Modelos A e B):
    * Exiba o gráfico das métricas ROC AUC e Accuracy calculadas no treinamento ao longo das épocas para o modelos A e B
    * Calcule e exiba as métricas no conjunto de teste usando o cada modelo</br>
    3.1 (2,0 pt) **Conclua** sobre os resultados obtidos nos seus experimentos, conforme descrito abaixo no notebook.</br></br>

4. (3,0 pt) **Fine-tuning** dos Modelo B com os dados em inglês, treinando por 15 épocas e batch_size=20, com Adam e taxa de aprendizado 0.00001 (sem decaimento):
    1. Modelo Fine Tuning versão 1: realize ajuste fino, congelando a camada de embeddings,
    2. Modelo Fine Tuning versão 2: realize ajuste fino, congelando a camada convolucional e a camada LSTM.
        
    * exibir a função de custo ao longo das épocas de treinamento para cada modelo,
    * calcular e exibir métricas dos modelos após fine-tuning usando o conjunto de teste em ingles (textos tokenizados)</br>
    4.1 (1,0 pt) **Conclua** sobre os resultados obtidos nos seus experimentos, conforme descrito abaixo no notebook.</br></br>

5. **Bônus:** (+1 ponto extra)
    * *Análise visual das características*: visualize scatterplots com os 2 dimensões obtidos com o método tSNE as classes dos exemplos atribuídas com cores ou marcadores diferentes:
        1. scatterplot com projeção tSNE do conjunto de teste referente ao embedding pré-treinado em português,
        1. scatterplot com projeção tSNE da última camada densa de 256 dimensões extraído do conjunto de teste tokenizado em português referente ao *Modelo B*,
        1. scatterplot com projeção tSNE da última camada densa de 256 dimensões extraído do conjunto de teste em inglês (tokens) referente ao *Modelo B* após ajuste fino versão 2.
    * Escreva comentários com suas conclusões sobre a análise visual.

## Preparação

### 1. Carregar bibliotecas e modelo pré-treinado SentenceTransformer

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from numpy.random import seed

from tensorflow.random import set_seed
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#### Verifique a versão do Tensorflow, recomendamos a 2.17.
OBS: o código pode ter problemas com a versão 2.18

In [ ]:
print(tf.__version__)

2.17.0


In [ ]:
# baixar modelo codificador de setenças
model = SentenceTransformer('stsb-xlm-r-multilingual')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### 2. Carregar base de dados e obter amostra

In [ ]:
!gdown 1ltAAPcaPD1epgs-F9i-nhaF350DPPrGK

# ler base de dados
df_orig = pd.read_csv("imdb-reviews-pt-br.csv", delimiter=',', engine='python')

Downloading...
From (original): https://drive.google.com/uc?id=1ltAAPcaPD1epgs-F9i-nhaF350DPPrGK
From (redirected): https://drive.google.com/uc?id=1ltAAPcaPD1epgs-F9i-nhaF350DPPrGK&confirm=t&uuid=434b70d8-ac82-44c9-a573-5e79d12a1727
To: /content/imdb-reviews-pt-br.csv
100% 127M/127M [00:02<00:00, 48.4MB/s]


In [ ]:
# obter amostra
df = df_orig.sample(12000).reset_index(drop=True)
df.sentiment.value_counts()

,count
sentiment,
pos,6018
neg,5982


### 3. Separar dados em treinamento e teste e preparar rótulos

Iremos também verificar o tamanho das strings em cada instancia, para considerar apenas um número de caracteres proporcional ao percentil 80.

In [ ]:
# rotulos
y = np.array((df.sentiment=='pos').astype(int))

X_train_txt_en, X_test_txt_en, y_train_en, y_test_en= train_test_split(df.text_en.values, y, test_size=0.25, random_state=51)
X_train_txt_pt, X_test_txt_pt, y_train_pt, y_test_pt= train_test_split(df.text_pt.values, y, test_size=0.25, random_state=51)

sum(y_train_en!=y_train_pt)

0

In [ ]:
# tamanhos das strings - usaremos o percentil 80 como tamanho da entrada máximo
num_words = df.text_pt.apply(lambda x: len(str(x).split()))
max_length = num_words.max()
percentile_80 = num_words.quantile(0.80)
percentile_95 = num_words.quantile(0.95)

print(f"percentil 80.0: {percentile_80}")
print(f"percentil 95.0: {percentile_95}")
print(f"Maximo: {max_length}")

percentil 80.0: 312.0
percentil 95.0: 538.0
Maximo: 936


### 4. Geração dos embeddings

Utilizar o encoder carregado para obter os embeddings para treinamento e teste em Português e Inglês, separadamente.

Exemplo de uso: `model.encode(texto, batch_size=16, show_progress_bar=True)`

In [ ]:
X_train_en = model.encode(X_train_txt_en, batch_size=16, show_progress_bar=True)
# ...

5. Tokenização do texto

Definiremos um número máximo de palavras e um tamanho máximo de sequência com base nos dados em Português, conforme abaixo.

In [ ]:
max_words = 5000  # Numero maximo de palavras
max_sequence_length = int(percentile_80)  # Tamanho maximo da sequencia igual ao percentil 80

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train_txt_pt)

sequences_train = tokenizer.texts_to_sequences(X_train_txt_pt)
sequences_test = tokenizer.texts_to_sequences(X_test_txt_pt)

# realizar padding para igualar sentencas
X_train_tok_pt = pad_sequences(sequences_train, maxlen=max_sequence_length)
X_test_tok_pt = pad_sequences(sequences_test, maxlen=max_sequence_length)

In [ ]:
## realizar o mesmo procedimento para os textos em ingles
# use os mesmos parametros de maximo de palavras e sequencia

---

## Tarefas

Definimos os modelos e iniciamos as tarefas. O código abaixo é uma referência para auxiliar na resolução, complete/altere conforme necessário.

In [ ]:
# DICA: use keras.layers.CAMADA para montar as arquiteturas

# definir modelo A
def model_A(input_dim):
    input_data = keras.layers.Input(shape=(input_dim,))
    # ...
    output = keras.layers.Dense(1, activation='sigmoid')(x)
    dnn = keras.models.Model(input_data, output)
    return dnn

In [ ]:
# definir modelo B
def model_B(input_dim, max_words, embedding_dim=386, code_dim=256):
    input_layer = keras.layers.Input(shape=(input_dim,))
    embedding_layer = keras.layers.Embedding(max_words, embedding_dim, name='embedding')(input_layer)
    # ...
    output_layer = keras.layers.Dense(1, activation='sigmoid')(dropout_layer)
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
epochs = 15
batch_size = 20
lrate = 0.0007

---

### 1. (1,0 pt) Modelo A: classificador de sentimento em *portugues* usando embedding pré-treinado

In [ ]:
seed(1)
set_seed(2)

def scheduler(epoch, lr):
    return float(lr * tf.math.exp(-0.09))

callbacklr = tf.keras.callbacks.LearningRateScheduler(scheduler)

# instanciar, compilar e treinar modelo A

---

### 2. (1,0 pt) Modelo B: classificador de sentimento em *português* usando tokenizador e rede CNN+LSTM

In [ ]:
seed(1)
set_seed(2)

# instanciar, compilar e treinar modelo B

---

### 3. (2,0 pt) Avaliação dos modelos: métricas durante treinamento e avaliação das métricas no conjunto de teste de ambos os modelos

In [ ]:
## plot métricas durante treinamento dos modelos A e B

In [ ]:
## avaliacao no conjunto de teste dos modelos A e B treinados
# exibir os valores obtidos

### 3.1 (2,0 pt) Escreva conclusões sobre a comparação entre os modelos A e B em termos:
- das métricas computadas ao longo do treinamento
- conclua sobre os resultados da generalização dos modelos e relacione esse resultado com a arquitetura de cada modelo (camadas utilizadas e número de parâmetros).

Escreva apenas um parágrafo para cada um dos items acima. As conclusões devem fazer sentido com relação aos resultados obtidos no seu notebook.

< suas conclusões aqui >
- **Métricas de treinamento**:
- **Generalização e arquitetura**:

---

### 4. (3,0 pt) Fine-tuning do Modelo B usando os dados em **ingles**

O modelo foi treinado com dados em português e agora queremos realizar transferência de aprendizado, realizando ajuste fino dos parâmetros para que possa classificar texto em inglês.

Serão feitos dois experimentos, ambos a partir dos pesos do modelo B, usando os tokens extraídos dos textos em ingles:

1. Modelo B FT v1: congele a camada de embedding (ou seja setando essa camada como `trainable=False`), e permita que todas as demais possam se adaptar.
2. Modelo B FT v2: congele apenas a camada convolucional e a camada LSTM, e permita que todas as demais (inclusive a camada de embedding, anterior a essas) possam se adaptar.

Após o treinamento compare ambos modelos exibindo:
- um gráfico com a loss ao longo do treinamento
- um gráfico com as métricas obtidas ao longo do treinamento
- as métricas (numéricas) obtidas no conjunto de teste

In [ ]:
## Experimentos de Fine-tuning do Modelo B

In [ ]:
# Cria novo modelo copiando a arquitetura e pesos anteriores
# ajusta camadas a serem treinadas ou congeladas
# realizar finetuning com learning rate fixo = 0.0001

In [ ]:
# visualiza loss de treinamento dos modelos

In [ ]:
# visualiza métricas de treinamento dos modelos

In [ ]:
# avaliacao e comparacao com dados de teste

#### 5.1 (1,0 pt) Conclusões sobre o Fine-tuning realizado

Conclua sobre o fine-tuning realizado, comparando os dois modelos abordando:
- a loss e métricas computadas ao longo do treinamento para cada modelo
- as métricas obtidas com os dados de teste
- compare os modelos em termos de sua capacidade de generalização, indicando as razões de terem generalizado melhor ou pior.

As conclusões devem fazer sentido com relação aos resultados obtidos no seu notebook. Escreva um ou dois parágrafos com suas conclusões.

_< suas conclusões >_

---
## Bônus (+1pt)

### Análise visual e conclusões

A partir dos espaços de características dos dados de teste usando tSNE em duas dimensões:

a. Espaço embedding original dos dados de teste em portugues,<br>
b. Espaço embedding Modelo B dos dados de teste em português,<br>
c. Espaço embedding Modelo B ajustado para inglês (segunda versão do finetuning) dos dados de teste em ingles.

Tarefas:
1. Plote os espaços projetados em 2D pelo TNSE usando os parametros definidos no exemplo abaixo. Utilize cores e/ou formatos de pontos no gráfico de forma a diferenciar os pontos de cada classe.

2. Escreva um parágrafo com conclusões sobre essa análise visual do espaço embedding/características aprendidas. As conclusões tem que fazer sentido com os plots exibidos.

_<suas conclusões>_

In [ ]:
from sklearn.manifold import TSNE

#parametros TSNE
#TSNE(n_components=2, learning_rate='auto', init='pca', perplexity=6)

In [ ]:
# plot dos espaços em 2D